# Transformers and Pipelines test on DatasetArray object

In this notebook we check the `caits.transformers` and Sklearn Pipelines consisting of `caits.transformers`.


## Importing libraries

In [1]:
import pandas as pd
from caits.filtering import filter_butterworth
from caits.fe import mean_value, std_value, stft, istft, melspectrogram
from caits.dataset import CoreArray, DatasetArray
from caits.transformers import (
    FunctionTransformer,
    FunctionTransformer2D,
    FeatureExtractorSignal,
    FeatureExtractorSpectrum,
    FeatureExtractorScalar,
    SlidingWindow
)

## Dataset loading

For this notebook, we will use the data/AirQuality.csv dataset.

In [2]:
data = pd.read_csv("data/AirQuality.csv", sep=";", decimal=",")
data_X = data.iloc[:, 2:-4]
data_X = data_X.fillna(data_X.mean())
data_y = data.iloc[:, -4:-2]
data_y = data_y.fillna(data_y.mean())

In [3]:
data_X_vals = data_X.values
data_X_axis_names = {"axis_1": data_X.columns}
data_y_vals = data_y.values
data_y_axis_names = {"axis_1": data_y.columns}
data_X = CoreArray(values=data_X_vals, axis_names=data_X_axis_names)
data_y = CoreArray(values=data_y_vals, axis_names=data_y_axis_names)
datasetArrayObj = DatasetArray(data_X, data_y)

In [4]:
datasetArrayObj.X

                CO(GT)         PT08.S1(CO)             NMHC(GT)            C6H6(GT)      PT08.S2(NMHC)            NOx(GT)  \
   0               2.6              1360.0                150.0                11.9             1046.0              166.0  
   1               2.0              1292.0                112.0                 9.4              955.0              103.0  
   2               2.2              1402.0                 88.0                 9.0              939.0              131.0  
   3               2.2              1376.0                 80.0                 9.2              948.0              172.0  
   4               1.6              1272.0                 51.0                 6.5              836.0              131.0  
 ...               ...                 ...                  ...                 ...                ...                ...  
9466  -34.207523778989  1048.9900609169606  -159.09009297851875  1.8656834455487867  894.5952762637597  168.6169712514695  
9467  -

## FunctionTransformer

This transformer is mainly used for transforming the `X` attribute of the `DatasetArray` object into a `CaitsArray`s with the shape maintained.

We test the `caits.transformer.FunctionTransformer` using the `caits.fe.filter_butterworth` function.


In [5]:
functionTransformer = FunctionTransformer(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)
transformedArray = functionTransformer.fit_transform(datasetArrayObj)

In [6]:
datasetArrayObj.X

                CO(GT)         PT08.S1(CO)             NMHC(GT)            C6H6(GT)      PT08.S2(NMHC)            NOx(GT)  \
   0               2.6              1360.0                150.0                11.9             1046.0              166.0  
   1               2.0              1292.0                112.0                 9.4              955.0              103.0  
   2               2.2              1402.0                 88.0                 9.0              939.0              131.0  
   3               2.2              1376.0                 80.0                 9.2              948.0              172.0  
   4               1.6              1272.0                 51.0                 6.5              836.0              131.0  
 ...               ...                 ...                  ...                 ...                ...                ...  
9466  -34.207523778989  1048.9900609169606  -159.09009297851875  1.8656834455487867  894.5952762637597  168.6169712514695  
9467  -

In [7]:
transformedArray.X

                  CO(GT)         PT08.S1(CO)             NMHC(GT)            C6H6(GT)       PT08.S2(NMHC)             NOx(GT)  \
   0   2.600272319669329  1360.0048242090925   150.01459442014138    11.9003076656988  1045.9967266517287  165.99348725991996  
   1  -1.420593185721463  1349.2114462959894   114.21039387234133   9.880820965996692   971.8875957681379  121.05917607668276  
   2  2.1179588334242263  1366.2579340007192    90.11432905570959   9.048350444158793   942.9659888266447  131.59650643638588  
   3  7.2455165546245945  1359.5097783999854    73.37218810168771   8.493980117895351   920.3551721055646    154.425981261632  
   4  1.7061240496375671  1290.0090330180367    55.14793984613857   6.920656043476639   850.6661770831126  132.42904581122934  
 ...                 ...                 ...                  ...                 ...                 ...                 ...  
9466  -34.20752377898899  1048.9900609169604   -159.0900929785187  1.8656834455487856   894.59527626375

In [8]:
datasetArrayObj.y

                     RH                  AH  
   0               48.9              0.7578  
   1               47.7              0.7255  
   2               54.0              0.7502  
   3               60.0              0.7867  
   4               59.6              0.7888  
 ...                ...                 ...  
9466  39.48537992946458  -6.837603644330447  
9467  39.48537992946458  -6.837603644330447  
9468  39.48537992946458  -6.837603644330447  
9469  39.48537992946458  -6.837603644330447  
9470  39.48537992946458  -6.837603644330447  

CoreArray with shape (9471, 2)

In [9]:
transformedArray.y

                     RH                  AH  
   0               48.9              0.7578  
   1               47.7              0.7255  
   2               54.0              0.7502  
   3               60.0              0.7867  
   4               59.6              0.7888  
 ...                ...                 ...  
9466  39.48537992946458  -6.837603644330447  
9467  39.48537992946458  -6.837603644330447  
9468  39.48537992946458  -6.837603644330447  
9469  39.48537992946458  -6.837603644330447  
9470  39.48537992946458  -6.837603644330447  

CoreArray with shape (9471, 2)

# FeatureExtractor

This transformer is mainly used for extracting single values per column or per row (if axis=1) for each instance of `DatasetArray.X`.

We test the `caits.transformer.FeatureExtractor` using the `caits.fe.mean_value` and `caits.fe.std_value`.

In [10]:
featureExtractor = FeatureExtractorSignal([
    {
        "func": mean_value,
        "params": {}
    },
    {
        "func": std_value,
        "params": {
            "ddof": 0
        }
    }
])

In [11]:
tmp = featureExtractor.fit_transform(datasetArrayObj)
tmp

DatasetArray object with 2 instances.

In [12]:
tmp.X

                       CO(GT)         PT08.S1(CO)             NMHC(GT)            C6H6(GT)      PT08.S2(NMHC)             NOx(GT)  \
mean_value   -34.207523778989  1048.9900609169606  -159.09009297851875  1.8656834455487865  894.5952762637597   168.6169712514695  
 std_value  77.18426094286016  327.82412536597025    138.9378182970468    41.1282131087734  340.2485424943651  255.86616950626888  

                 PT08.S3(NOx)             NO2(GT)        PT08.S4(NO2)        PT08.S5(O3)                   T  
mean_value  794.9901677888212   58.14887250187026  1391.4796409105484  975.0720316340708   9.778305012290264  
 std_value  320.0327052589704  126.16742509610036  464.36495185057805  454.1555648716221  42.940525662335475  

CoreArray with shape (2, 11)

In [13]:
tmp.y

                     RH                  AH  
   0               48.9              0.7578  
   1               47.7              0.7255  
   2               54.0              0.7502  
   3               60.0              0.7867  
   4               59.6              0.7888  
 ...                ...                 ...  
9466  39.48537992946458  -6.837603644330447  
9467  39.48537992946458  -6.837603644330447  
9468  39.48537992946458  -6.837603644330447  
9469  39.48537992946458  -6.837603644330447  
9470  39.48537992946458  -6.837603644330447  

CoreArray with shape (9471, 2)

## FeatureExtractor2D

This transformer is mainly used for extracting 2D features per column of `DatasetArray.X`.

We test this using the `caits.fe.melspectrogram` and `caits.fe.stft`.
Applying each of these functions will transform the `CaitsArray` of `DatasetArray.X` into a 3D `CaitsArray`.


In [14]:
featureExtractor2D = FeatureExtractorSpectrum(melspectrogram, n_fft=100, hop_length=10)
tmp = featureExtractor2D.fit_transform(datasetArrayObj)

/home/AIoT03/PycharmProjects/crossai-ts/caits/fe/_spectrum.py:127: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = mel_filter(sr=sr, n_fft=n_fft, **kwargs)


In [15]:
tmp.X.shape

(11, 128, 948)

In [16]:
featureExtractor2D = FeatureExtractorSpectrum(stft, n_fft=100, hop_length=10)
tmp1 = featureExtractor2D.fit_transform(datasetArrayObj)

In [17]:
tmp1.X.iloc[:, 0, 0]

       CO(GT)  (-203.00489003333593+0j)
  PT08.S1(CO)   (31352.259235229427+0j)
     NMHC(GT)    (2103.365810117917+0j)
     C6H6(GT)    (183.7569916168304+0j)
PT08.S2(NMHC)    (21178.65692255352+0j)
      NOx(GT)    (2830.009535623774+0j)
 PT08.S3(NOx)   (33108.013756721906+0j)
      NO2(GT)   (2096.0012268926475+0j)
 PT08.S4(NO2)   (37830.104266490875+0j)
  PT08.S5(O3)   (22635.975719316466+0j)
            T    (263.0239320519568+0j)

CoreArray with shape (11,)

## FunctionTransformer2D

This is mainly used to inverse the `featureExtractor2D` process. So, if `DatasetList.X` is a `CaitsArray` object, it will be
transformed in a `CaitsArray`.

To test this we use the `caits.fe.istft` on the transformed `DatasetArray` object using `caits.fe.stft`.

In [18]:
functionTransformer = FunctionTransformer2D(istft, n_fft=100, hop_length=10)
tmp2 = functionTransformer.fit_transform(tmp1)

In [19]:
tmp2.X

                  CO(GT)         PT08.S1(CO)             NMHC(GT)            C6H6(GT)       PT08.S2(NMHC)             NOx(GT)  \
   0  2.5999999999999996  1360.0000000000002   150.00000000000003  11.900000000000002  1046.0000000000002               166.0  
   1                 2.0  1291.9999999999998   111.99999999999999                 9.4   954.9999999999998               103.0  
   2   2.199999999999998              1402.0                 88.0   9.000000000000004               939.0               131.0  
   3  2.2000000000000033  1376.0000000000005                 80.0                 9.2   948.0000000000001  172.00000000000006  
   4   1.600000000000001              1272.0    51.00000000000002   6.500000000000001   836.0000000000001               131.0  
 ...                 ...                 ...                  ...                 ...                 ...                 ...  
9465  -34.20752377898901  1048.9900609169608  -159.09009297851878  1.8656834455487872   894.59527626375

## SlidingWindow

This is used for performing the sliding window process in each instance of the `DatasetArray` object.

The final windows will be appended in a single `DatasetList` object.

In [20]:
slidingWindow = SlidingWindow(window_size=20, overlap=5)
tmp = slidingWindow.fit_transform(datasetArrayObj)

In [21]:
tmp

DatasetList object with 631 instances.

In [22]:
tmp.X[0]

    CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
 0     2.6       1360.0     150.0      11.9         1046.0    166.0  
 1     2.0       1292.0     112.0       9.4          955.0    103.0  
 2     2.2       1402.0      88.0       9.0          939.0    131.0  
 3     2.2       1376.0      80.0       9.2          948.0    172.0  
 4     1.6       1272.0      51.0       6.5          836.0    131.0  
...     ...          ...       ...       ...            ...      ...  
15     2.2       1351.0      87.0       9.5          960.0    129.0  
16     1.7       1233.0      77.0       6.3          827.0    112.0  
17     1.5       1179.0      43.0       5.0          762.0     95.0  
18     1.6       1236.0      61.0       5.2          774.0    104.0  
19     1.9       1286.0      63.0       7.3          869.0    146.0  

    PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T  
 0        1056.0    113.0        1692.0       1268.0  13.6  
 1        1174.0     92.0        15

In [23]:
tmp.y

                     RH                  AH  
   0               48.9              0.7578  
   1               47.7              0.7255  
   2               54.0              0.7502  
   3               60.0              0.7867  
   4               59.6              0.7888  
 ...                ...                 ...  
9466  39.48537992946458  -6.837603644330447  
9467  39.48537992946458  -6.837603644330447  
9468  39.48537992946458  -6.837603644330447  
9469  39.48537992946458  -6.837603644330447  
9470  39.48537992946458  -6.837603644330447  

CoreArray with shape (9471, 2)

## SklearnWrapper

In [24]:
from sklearn.preprocessing import StandardScaler
from caits.transformers import SklearnWrapper, DatasetToArray, ArrayToDataset

dataFlatten = DatasetToArray(flatten=False)
scaler = SklearnWrapper(StandardScaler)
dataInverseFlatten = ArrayToDataset(shape=(9471, 11), flattened=False)


In [25]:
tmp_conv = dataFlatten.fit_transform(datasetArrayObj)
tmp_scaled = scaler.fit_transform(tmp_conv)
tmp_back = dataInverseFlatten.fit_transform(tmp_scaled)

In [26]:
tmp_scaled.X

                        0                   1                   2                       3                     4                      5  \
   0  0.47687861915575985   0.948709734940465  2.2246649383660912     0.24397647736149058   0.44498272535215255  -0.010227890840431627  
   1   0.4691050135958882  0.7412814380630237  1.9511612914414171     0.18319095299678864    0.1775311755736342   -0.25645035988183595  
   2   0.4716962154491788  1.0768272124235905  1.7784221460153071     0.17346526909843632   0.13050672726092769   -0.14701815141898958  
   3   0.4716962154491788  0.9975163930292748  1.7208424308732704     0.17832811104761245    0.1569579794368251   0.013221868115892594  
   4   0.4639226098893071  0.6802731154520116  1.5121159634833876     0.11267974473373435  -0.17221315875212054   -0.14701815141898958  
 ...                  ...                 ...                 ...                     ...                   ...                    ...  
9466                  0.0               

## ColumnTransformer

In [27]:
from caits.filtering import filter_median_gen
from caits.transformers import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from caits.properties import magnitude_signal

pipe_filter = Pipeline(
    [
        ("median", FunctionTransformer(filter_median_gen, window_size=20)),
        ("butterworth", FunctionTransformer(filter_butterworth, fs=10, filter_type='highpass', cutoff_freq=2))
    ]
)

pipe_scaler = Pipeline(
    [
        ("flatten", DatasetToArray(flatten=False)),
        ("scaler", SklearnWrapper(StandardScaler)),
        ("unflatten", ArrayToDataset(shape=(9471,13), flattened=False)),
    ]
)

mag_tr = FeatureExtractorSignal(
    [
        {
            "func": magnitude_signal,
            "params": {
                "axis": 1
            }
        }
    ], axis=1
)

column_tr1 = ColumnTransformer(
    [
        ("filter_acc_x_gyr_x", pipe_filter, ["NO2(GT)", "CO(GT)"], ["new_NO2", "new_CO"]),
        ("filter_acc_y_gyr_y", pipe_filter, ["T", "NMHC(GT)"], ["new_T", "new_NMHC"]),
    ],
    unify=False
)

column_tr2 = ColumnTransformer(
    [
        ("scale_acc_x_acc_y_acc_z", pipe_scaler, ["NO2(GT)", "CO(GT)"], ["scaled_NO2", "scaled_CO"]),
    ],
    unify=True
)

column_tr3 = ColumnTransformer(
    [
        ("mag_calc_1", mag_tr, ["NO2(GT)", "CO(GT)"], ["mag_NO2_CO"]),
        ("mag_calc_2", mag_tr, ["T", "NMHC(GT)"], ["mag_T_NHMC"]),
        ("mag_calc_3", mag_tr, ["scaled_NO2", "scaled_CO"], ["mag_scaled_NO2_CO"]),
    ],
    unify=True
)

final_pipe = Pipeline(
    [
        ("filter", column_tr1),
        ("scale", column_tr2),
        ("mag", column_tr3),
    ]
)


In [28]:
final_data = final_pipe.fit_transform(datasetArrayObj)

In [29]:
final_data

DatasetArray object with 9471 instances.

In [30]:
final_data.X.shape

(9471, 16)

In [31]:
datasetArrayObj.X.values

array([[-4.18082444e-04,  1.36000000e+03,  4.33262584e-05, ...,
         1.69200000e+03,  1.26800000e+03,  4.33262584e-05],
       [-1.78241045e-03,  1.29200000e+03,  1.01911635e-01, ...,
         1.55900000e+03,  9.72000000e+02,  1.01911635e-01],
       [ 5.65202666e-04,  1.40200000e+03, -1.05458940e-01, ...,
         1.55500000e+03,  1.07400000e+03, -1.05458940e-01],
       ...,
       [-1.32432902e-15,  1.04899006e+03, -7.99553627e-16, ...,
         1.39147964e+03,  9.75072032e+02, -7.99553627e-16],
       [ 8.33532814e-15,  1.04899006e+03, -3.94561262e-16, ...,
         1.39147964e+03,  9.75072032e+02, -3.94561262e-16],
       [ 4.70383955e-15,  1.04899006e+03,  5.12639240e-16, ...,
         1.39147964e+03,  9.75072032e+02,  5.12639240e-16]])

In [32]:
final_data.X.values

array([[-4.18082444e-04,  1.36000000e+03,  4.33262584e-05, ...,
         5.91257862e-04,  6.12725823e-05,  8.72312714e-05],
       [-1.78241045e-03,  1.29200000e+03,  1.01911635e-01, ...,
         2.52070904e-03,  1.44124817e-01,  3.71945995e-04],
       [ 5.65202666e-04,  1.40200000e+03, -1.05458940e-01, ...,
         7.99317276e-04,  1.49141464e-01,  1.17965533e-04],
       ...,
       [-1.32432902e-15,  1.04899006e+03, -7.99553627e-16, ...,
         1.87288407e-15,  1.13073958e-15,  1.62428149e-08],
       [ 8.33532814e-15,  1.04899006e+03, -3.94561262e-16, ...,
         1.17879341e-14,  5.57993888e-16,  1.62428170e-08],
       [ 4.70383955e-15,  1.04899006e+03,  5.12639240e-16, ...,
         6.65223368e-15,  7.24981366e-16,  1.62428162e-08]])

In [33]:
final_data.X

                       CO(GT)         PT08.S1(CO)                NMHC(GT)            C6H6(GT)      PT08.S2(NMHC)            NOx(GT)  \
   0   -0.0004180824437786498              1360.0   4.332625842047924e-05                11.9             1046.0              166.0  
   1   -0.0017824104543699292              1292.0      0.1019116350952852                 9.4              955.0              103.0  
   2    0.0005652026664373431              1402.0    -0.10545894039600734                 9.0              939.0              131.0  
   3    0.0034686667824226016              1376.0    -0.01740602384915324                 9.2              948.0              172.0  
   4    0.0011188865278696758              1272.0     0.05760856806207641                 6.5              836.0              131.0  
 ...                      ...                 ...                     ...                 ...                ...                ...  
9466  -1.1956243924209014e-14  1048.9900609169606   7.7623555

In [34]:
final_data.X.shape, final_data.y.shape

((9471, 16), (9471, 2))

## Test statistical

In [35]:
from caits.fe import central_moments

In [36]:
datasetArrayObj.apply(central_moments)

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 4.59248486e+01,  1.07468657e+05,  1.64472880e+01,
         1.69152991e+03,  1.15769071e+05,  6.54674967e+04,
         1.02420932e+05,  4.59248486e+01,  2.15634809e+05,
         2.06257277e+05,  1.64472880e+01],
       [-1.59902008e+01, -6.10085362e+07,  2.76400208e+01,
        -3.15527266e+05, -3.14383651e+07,  1.39051198e+07,
        -1.26862339e+07, -1.59902008e+01, -1.25313263e+08,
        -3.26607390e+06,  2.76400208e+01],
       [ 1.58453947e+05,  1.03261810e+11,  5.25228680e+04,
         6.42297451e+07,  7.28234886e+10,  1.95391795e+10,
  

In [37]:
central_mom_transformer = FeatureExtractorScalar(
    [
        {
            "func": central_moments
        }
    ],
    to_dataset=True
)

central_mom_transformer.fit_transform(datasetArrayObj)

DatasetArray object with 5 instances.

In [38]:
from caits.fe import max_value

datasetArrayObj.apply(max_value)

array([  76.77615256, 2040.        ,   59.85697746,   63.7       ,
       2214.        , 1479.        , 2683.        ,   76.77615256,
       2775.        , 2523.        ,   59.85697746])

In [39]:
max_tr = FeatureExtractorScalar(
    [
        {
            "func": max_value
        }
    ],
)

max_tr.fit_transform(datasetArrayObj).X

                      CO(GT)  PT08.S1(CO)           NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
max_value  76.77615255607856       2040.0  59.85697745978721      63.7         2214.0   1479.0  

           PT08.S3(NOx)            NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)                  T  
max_value        2683.0  76.77615255607856        2775.0       2523.0  59.85697745978721  

CoreArray with shape (1, 11)

In [40]:
from caits.fe import mfcc_mean

datasetArrayObj.apply(mfcc_mean, n_mfcc=5)

array([[ 5.16266952e+02,  5.50944468e+02,  5.45717523e+02,
         5.47058442e+02,  5.56117943e+02,  5.40888168e+02,
         5.29344071e+02,  5.47444093e+02,  5.65268994e+02,
         5.63746201e+02,  5.37861391e+02,  5.33177682e+02,
         5.43988159e+02,  5.68706143e+02,  5.69963015e+02,
         5.61020379e+02,  5.51937644e+02,  5.26563661e+02,
         5.08563015e+02],
       [ 4.54674740e+01,  4.05813861e+01,  3.96347487e+01,
         2.98971440e+01,  3.04124800e+01,  3.38017753e+01,
         1.91315060e+01,  1.88360829e+01,  2.57814788e+01,
         2.51212436e+01,  2.94505552e+01,  3.91609732e+01,
         4.41966898e+01,  3.89025175e+01,  3.63648152e+01,
         3.95535933e+01,  4.25405262e+01,  3.79951530e+01,
         4.11370224e+01],
       [ 1.51386209e+00,  7.09165584e+00,  8.00676696e+00,
         1.26804795e+01,  1.78734055e+01,  1.55251002e+01,
         1.31492541e+01,  1.64198931e+01,  2.25026190e+01,
         1.99058561e+01,  8.88387701e+00,  5.20527222e+00,
    

In [41]:
from caits.fe import dominant_frequency

datasetArrayObj.apply(dominant_frequency, fs=50)

array([1.18625277e+01, 2.08531306e+00, 1.11498258e+01, 3.37873509e-01,
       2.08531306e+00, 5.27927357e-03, 2.08531306e+00, 1.18625277e+01,
       5.27927357e-03, 4.16534685e+00, 1.11498258e+01])

In [42]:
from caits.fe import spectral_kurtosis
datasetArrayObj.apply(spectral_kurtosis, fs=100)

array([1.92457255, 3.60532522, 1.87857391, 3.27548866, 3.34208442,
       2.81176936, 3.78551685, 1.92457255, 3.54338823, 3.51919915,
       1.87857391])

In [43]:
from caits.fe import (
mean_value,
std_value,
variance_value,
kurtosis_value,
dominant_frequency,
max_value,
average_power,
min_value,
energy,
crest_factor,
sample_skewness,
delta,
envelope_energy_peak_detection,
rms_max,
rms_min,
rms_value,
rms_mean,
zcr_max,
zcr_min,
zcr_value,
zcr_mean,
spectral_bandwidth,
spectral_std,
spectral_values,
spectral_kurtosis,
spectral_slope,
spectral_spread,
spectral_rolloff,
spectral_skewness,
spectral_centroid,
spectral_decrease,
spectral_flatness,
median_value,
signal_length,
max_possible_amplitude,
underlying_spectral
)

scalar_tr = FeatureExtractorScalar(
    [
        {
            "func": mean_value
        },
        {
            "func": std_value
        },
        {
            "func": variance_value
        },
        {
            "func": kurtosis_value
        },
        {
            "func": dominant_frequency,
            "params": {
                "fs": 100
            }
        },
        {
            "func": max_value
        },
        {
            "func": crest_factor
        },
        {
            "func": min_value
        },
        {
            "func": energy
        },
        {
            "func": crest_factor
        },
        {
            "func": average_power
        },
        {
            "func": sample_skewness
        },
        {
            "func": rms_mean,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": rms_value,
        },
        {
            "func": rms_max,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": rms_min,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": zcr_value
        },
        {
            "func": zcr_max,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": zcr_min,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": zcr_mean,
            "params": {
                "frame_length": 20,
                "hop_length": 10
            }
        },
        {
            "func": spectral_bandwidth,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_std,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_kurtosis,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_slope,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_rolloff,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_skewness,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_centroid,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_decrease,
            "params": {
                "fs": 100
            }
        },
        {
            "func": spectral_flatness,
            "params": {
                "fs": 100
            }
        },
        {
            "func": median_value,
        },
        {
            "func": central_moments
        },
        {
            "func": delta,
            "params": {
                "width": 201,
                "order": 1
            }
        }
    ]
)

In [44]:
tmp = scalar_tr.fit_transform(datasetArrayObj)
tmp.X

(4735,) (4735, 11)


                                     CO(GT)         PT08.S1(CO)                NMHC(GT)               C6H6(GT)       PT08.S2(NMHC)               NOx(GT)  \
        mean_value   -7.783414738334147e-08  1048.9900609169606   3.146543898348442e-06     1.8656834455487865   894.5952762637597     168.6169712514695  
         std_value        6.776787483584214  327.82412536597025       4.055525607298952       41.1282131087734   340.2485424943651    255.86616950626888  
    variance_value        45.92484859766367   107468.6571719634       16.44728795145753     1691.5299135206803  115769.07066953977     65467.49669781071  
    kurtosis_value         72.1289979001016   5.940794012097449       191.1600347310216      19.44795762468884  2.4335840055080107    1.5588452730444633  
dominant_frequency       23.725055432372503  4.1706261218456335       22.29965156794425     0.6757470172104317  4.1706261218456335  0.010558547143912996  
               ...                      ...                 ...      

In [45]:
datasetArrayObj.apply(min_value)

array([-113.83163945, -200.        ,  -59.75381293, -200.        ,
       -200.        , -200.        , -200.        , -113.83163945,
       -200.        , -200.        ,  -59.75381293])

In [46]:
from caits.fe import central_moments

# datasetArrayObj.apply(central_moments)
centrals = central_mom_transformer.fit_transform(datasetArrayObj)
centrals

DatasetArray object with 5 instances.

In [47]:
centrals.X

                                CO(GT)         PT08.S1(CO)           NMHC(GT)            C6H6(GT)        PT08.S2(NMHC)             NOx(GT)  \
central_moments_0                  1.0                 1.0                1.0                 1.0                  1.0                 1.0  
central_moments_1                  0.0                 0.0                0.0                 0.0                  0.0                 0.0  
central_moments_2    45.92484859766367   107468.6571719634  16.44728795145753  1691.5299135206803   115769.07066953977   65467.49669781071  
central_moments_3  -15.990200833753557  -61008536.15455512  27.64002076846445  -315527.2656574346  -31438365.131765038  13905119.843258983  
central_moments_4    158453.9473067111  103261810185.10976  52522.86802603895   64229745.12087768    72823488593.61337  19539179493.091637  

                          PT08.S3(NOx)              NO2(GT)         PT08.S4(NO2)          PT08.S5(O3)                  T  
central_moments_0            